In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/GazeGaussian

In [ ]:
!nvidia-smi

In [ ]:
!python verify_enhanced_implementation.py

In [ ]:
!python verify_dit_mode.py

In [ ]:
import os
import shutil
from pathlib import Path

drive_checkpoint_dir = "/content/drive/MyDrive/GazeGaussian_checkpoints/gazegaussian"
drive_validation_dir = "/content/drive/MyDrive/GazeGaussian_checkpoints/validation_images"
os.makedirs(drive_checkpoint_dir, exist_ok=True)
os.makedirs(drive_validation_dir, exist_ok=True)

def save_checkpoint_to_drive():
    local_ckpt_dir = Path("work_dirs")
    if local_ckpt_dir.exists():
        for ckpt_file in local_ckpt_dir.glob("**/checkpoints/*.pth"):
            dest = Path(drive_checkpoint_dir) / ckpt_file.name
            shutil.copy(ckpt_file, dest)
            print(f"Saved {ckpt_file.name} to Google Drive")
        
        for result_file in local_ckpt_dir.glob("**/results/*.png"):
            dest = Path(drive_validation_dir) / result_file.name
            shutil.copy(result_file, dest)
            print(f"Saved {result_file.name} to Google Drive")

In [ ]:
meshhead_checkpoint = "/content/drive/MyDrive/GazeGaussian_checkpoints/meshhead/latest.pth"

if os.path.exists(meshhead_checkpoint):
    print(f"Using MeshHead checkpoint from Google Drive: {meshhead_checkpoint}")
    cmd = f"python train_gazegaussian.py --batch_size 1 --num_epochs 100 --save_freq 500 --load_meshhead_checkpoint {meshhead_checkpoint}"
elif os.path.exists("work_dirs/meshhead/checkpoints/latest.pth"):
    print("Using local MeshHead checkpoint")
    cmd = "python train_gazegaussian.py --batch_size 1 --num_epochs 100 --save_freq 500 --load_meshhead_checkpoint work_dirs/meshhead/checkpoints/latest.pth"
else:
    print("WARNING: No MeshHead checkpoint found!")
    print("Training from scratch (not recommended)")
    cmd = "python train_gazegaussian.py --batch_size 1 --num_epochs 100 --save_freq 500"

print(f"\nCommand: {cmd}")

In [ ]:
!{cmd}

In [ ]:
save_checkpoint_to_drive()
print("\nGazeGaussian training complete!")
print(f"Checkpoints saved to: {drive_checkpoint_dir}")
print(f"Validation images saved to: {drive_validation_dir}")

In [ ]:
latest_checkpoint = None
for ckpt in Path("work_dirs").glob("**/checkpoints/*.pth"):
    if "latest" in ckpt.name or "final" in ckpt.name:
        latest_checkpoint = str(ckpt)
        break

if latest_checkpoint:
    print(f"Generating validation images from: {latest_checkpoint}")
    !python generate_validation_images.py --checkpoint {latest_checkpoint} --num_samples 16 --output_dir validation_outputs
    
    if os.path.exists("validation_outputs"):
        for img in Path("validation_outputs").glob("*.png"):
            shutil.copy(img, drive_validation_dir)
        print(f"Validation images copied to: {drive_validation_dir}")
else:
    print("No checkpoint found for validation")

In [ ]:
from IPython.display import Image, display
import glob

validation_images = glob.glob("validation_outputs/*.png")[:8]
print(f"Displaying {len(validation_images)} validation images:\n")

for img_path in validation_images:
    print(f"\n{os.path.basename(img_path)}:")
    display(Image(filename=img_path, width=800))

In [ ]:
print("\n" + "="*80)
print("TRAINING COMPLETE!")
print("="*80)
print(f"\nAll files saved to Google Drive:")
print(f"  Checkpoints: {drive_checkpoint_dir}")
print(f"  Validation: {drive_validation_dir}")
print("\nActive Features:")
print("  ✓ DiT Neural Renderer with AdaLN-Zero")
print("  ✓ VAE Integration (latent compression)")
print("  ✓ Orthogonality Loss (disentangled control)")
print("  ✓ Intermediate Gaze Training (every 5 batches)")
print("="*80)